In [4]:
import Augmentor
from IPython.display import Image, display, HTML

from glob import glob
import pandas as pd
import os
import shutil

In [7]:
images = pd.read_csv('data/metadata_rgb_only.csv')

new_directory = 'data/augumented_data'
os.makedirs(new_directory, exist_ok=True)
os.makedirs(new_directory + '/tumor', exist_ok=True)
os.makedirs(new_directory + '/normal', exist_ok=True)

for image in images.iterrows():
    if image[1]['image'][-3:] == 'jpg' and image[1]['mode'] == 'RGB':
        if image[1]['class'] == 'tumor':
            source_path = os.path.join('data/Brain Tumor Data Set/Brain Tumor/', image[1]['image'])
            destination_path = os.path.join(new_directory + '/tumor', image[1]['image'])
            shutil.copy(source_path, destination_path)
        else:
            source_path = os.path.join('data/Brain Tumor Data Set/Healthy/', image[1]['image'])
            destination_path = os.path.join(new_directory + '/normal', image[1]['image'])
            shutil.copy(source_path, destination_path)



In [8]:
path_to_files = 'data/augumented_data/tumor/'

p = Augmentor.Pipeline(path_to_files)

p.flip_left_right(probability=0.5)
p.random_brightness(probability=0.5, min_factor=0.3, max_factor=0.6)

p.set_save_format
p.sample(3000)

Initialised with 2267 image(s) found.
Output directory set to data/augumented_data/tumor/output.

Processing <PIL.Image.Image image mode=RGB size=279x354 at 0x11F123D10>: 100%|██████████| 3000/3000 [00:06<00:00, 470.15 Samples/s]                     


In [9]:
path_to_files = 'data/augumented_data/normal/'

p = Augmentor.Pipeline(path_to_files)

p.flip_left_right(probability=0.5)
p.random_brightness(probability=0.5, min_factor=0.3, max_factor=0.6)

p.sample(3000)

Initialised with 2030 image(s) found.
Output directory set to data/augumented_data/normal/output.

Processing <PIL.Image.Image image mode=RGB size=173x201 at 0x11C32D650>: 100%|██████████| 3000/3000 [00:04<00:00, 673.42 Samples/s]                   
